In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.dates import DateFormatter
from collections import deque
import random

# Load and prepare the data
def load_traffic_data(data_path):
    # In a real implementation, you'd load from a file
    df = pd.read_csv(data_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

# Define the VSL environment
class VSLEnvironment:
    def __init__(self, data):
        self.data = data
        self.current_step = 0
        self.speed_limits = [60, 80, 100, 120, 140]  # Available speed limits in km/h
        
        # Define state space features
        self.state_features = ['vehicle_count', 'average_speed_kmh', 'vehicle_density_vpkm']
        
        # Normalize the features for better learning
        self.feature_means = data[self.state_features].mean()
        self.feature_stds = data[self.state_features].std()
        
    def reset(self):
        self.current_step = 0
        return self._get_state()
    
    def _get_state(self):
        if self.current_step >= len(self.data):
            return None
        
        current_data = self.data.iloc[self.current_step][self.state_features]
        # Normalize the state
        normalized_state = (current_data - self.feature_means) / self.feature_stds
        return normalized_state.values.astype(np.float32)  # Ensure float32 type
    
    def step(self, action):
        if self.current_step >= len(self.data) - 1:
            return None, 0, True, {}
        
        # Get current traffic conditions
        current_data = self.data.iloc[self.current_step]
        
        # Apply the selected speed limit
        selected_speed_limit = self.speed_limits[action]
        
        # Advance to the next step
        self.current_step += 1
        next_data = self.data.iloc[self.current_step]
        
        # Calculate reward based on traffic efficiency and safety
        reward = self._calculate_reward(current_data, next_data, selected_speed_limit)
        
        # Get new state
        new_state = self._get_state()
        
        # Check if episode is done
        done = self.current_step >= len(self.data) - 1
        
        return new_state, reward, done, {'speed_limit': selected_speed_limit}
    
    def _calculate_reward(self, current_data, next_data, speed_limit):
        # Traffic efficiency component: Reward higher flow rates
        flow_reward = next_data['flow_rate_vph'] / 1000  # Normalize
        
        # Safety component: Penalize if speed is much higher than the limit
        speed_compliance = max(0, 1 - max(0, next_data['average_speed_kmh'] - speed_limit) / 50)
        
        # Stability component: Reward lower variations in speed
        speed_stability = 1 / (1 + abs(next_data['average_speed_kmh'] - current_data['average_speed_kmh']))
        
        # Occupancy component: Reward balanced lane utilization
        occupancy_vars = np.var([
            next_data['occupancy_lane_1'], 
            next_data['occupancy_lane_2'], 
            next_data['occupancy_lane_3']
        ])
        occupancy_balance = 1 / (1 + occupancy_vars/1000)
        
        # Combined reward
        reward = (0.4 * flow_reward + 
                  0.3 * speed_compliance + 
                  0.2 * speed_stability + 
                  0.1 * occupancy_balance)
        
        return float(reward)  # Ensure float type

# Implement Deep Q-Network agent

# Previous VSLEnvironment remains the same as in the original code

class PPOAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        # Hyperparameters
        self.learning_rate = 0.0003
        self.gamma = 0.99  # Discount factor
        self.epsilon_clip = 0.2  # Clipping parameter
        self.c1 = 1.0  # Value loss coefficient
        self.c2 = 0.01  # Entropy coefficient
        self.epochs = 3  # Number of epochs per update
        
        # Memory for storing experiences
        self.memory = []
        
        # Actor-Critic Network
        self.actor = self._build_actor_network()
        self.critic = self._build_critic_network()
        
        # Optimizers
        self.actor_optimizer = keras.optimizers.Adam(learning_rate=self.learning_rate)
        self.critic_optimizer = keras.optimizers.Adam(learning_rate=self.learning_rate)
    
    def _build_actor_network(self):
        # Actor network for policy (action probabilities)
        inputs = layers.Input(shape=(self.state_size,))
        x = layers.Dense(64, activation='relu')(inputs)
        x = layers.Dense(64, activation='relu')(x)
        
        # Output layer with softmax for action probabilities
        outputs = layers.Dense(self.action_size, activation='softmax')(x)
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        return model
    
    def _build_critic_network(self):
        # Critic network for value function estimation
        inputs = layers.Input(shape=(self.state_size,))
        x = layers.Dense(64, activation='relu')(inputs)
        x = layers.Dense(64, activation='relu')(x)
        
        # Output a single value
        outputs = layers.Dense(1, activation='linear')(x)
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        return model
    
    def remember(self, state, action, reward, next_state, done, old_prob):
        # Store experiences with additional information for PPO
        self.memory.append((state, action, reward, next_state, done, old_prob))
    
    def act(self, state):
        # Predict action probabilities
        state = np.array([state], dtype=np.float32)
        action_probs = self.actor.predict(state, verbose=0)[0]
        
        # Sample action based on probabilities
        action = np.random.choice(self.action_size, p=action_probs)
        
        # Get log probability of the chosen action
        log_prob = np.log(action_probs[action])
        
        return action  # Return only the action index
    
    def train(self, batch_size):
        if len(self.memory) < batch_size:
            return
        
        # Prepare batches
        states = np.array([mem[0] for mem in self.memory], dtype=np.float32)
        actions = np.array([mem[1] for mem in self.memory])
        rewards = np.array([mem[2] for mem in self.memory], dtype=np.float32)
        next_states = np.array([mem[3] for mem in self.memory], dtype=np.float32)
        dones = np.array([mem[4] for mem in self.memory])
        old_log_probs = np.array([mem[5] for mem in self.memory])
        
        # Compute returns and advantages
        returns = np.zeros_like(rewards)
        advantages = np.zeros_like(rewards)
        
        # Compute returns and advantages
        values = self.critic.predict(states, verbose=0).flatten()
        next_values = self.critic.predict(next_states, verbose=0).flatten()
        
        for t in range(len(rewards)):
            # Compute TD error
            delta = rewards[t] + self.gamma * next_values[t] * (1 - dones[t]) - values[t]
            
            # Compute advantage
            advantages[t] = delta
            
            # Compute returns (GAE)
            returns[t] = rewards[t] + self.gamma * next_values[t] * (1 - dones[t])
        
        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-10)
        
        # Perform PPO update using a custom training function
        self._ppo_update(states, actions, advantages, returns, old_log_probs)
        
        # Clear memory after training
        self.memory.clear()
    
    def _ppo_update(self, states, actions, advantages, returns, old_log_probs):
        # Use GradientTape for explicit gradient computation
        with tf.GradientTape() as actor_tape, tf.GradientTape() as critic_tape:
            # Actor network computations
            current_probs = self.actor(states)
            
            # Compute log probabilities
            indices = tf.range(len(actions))
            action_masks = tf.one_hot(actions, self.action_size)
            current_log_probs = tf.math.log(tf.reduce_sum(current_probs * action_masks, axis=1) + 1e-10)
            
            # Compute probability ratio
            ratio = tf.exp(current_log_probs - tf.cast(old_log_probs, tf.float32))
            
            # Clipped surrogate objective
            surr1 = ratio * advantages
            surr2 = tf.clip_by_value(ratio, 1 - self.epsilon_clip, 1 + self.epsilon_clip) * advantages
            policy_loss = -tf.reduce_mean(tf.minimum(surr1, surr2))
            
            # Critic (value) loss
            critic_values = tf.squeeze(self.critic(states))
            value_loss = tf.reduce_mean(tf.square(returns - critic_values))
            
            # Entropy loss
            entropy_loss = -tf.reduce_mean(current_probs * tf.math.log(current_probs + 1e-10))
            
            # Composite losses
            total_actor_loss = policy_loss - self.c2 * entropy_loss
            total_critic_loss = self.c1 * value_loss
        
        # Compute gradients
        actor_grads = actor_tape.gradient(total_actor_loss, self.actor.trainable_variables)
        critic_grads = critic_tape.gradient(total_critic_loss, self.critic.trainable_variables)
        
        # Clip gradients to prevent exploding gradients
        max_grad_norm = 1.0
        actor_grads = [tf.clip_by_norm(grad, max_grad_norm) for grad in actor_grads]
        critic_grads = [tf.clip_by_norm(grad, max_grad_norm) for grad in critic_grads]
        
        # Apply gradients
        self.actor_optimizer.apply_gradients(zip(actor_grads, self.actor.trainable_variables))
        self.critic_optimizer.apply_gradients(zip(critic_grads, self.critic.trainable_variables))
        
        # Clear memory after training
        self.memory.clear()
    
    def save(self, actor_path, critic_path):
        self.actor.save_weights(actor_path)
        self.critic.save_weights(critic_path)
    
    def load(self, actor_path, critic_path):
        self.actor.load_weights(actor_path)
        self.critic.load_weights(critic_path)

# Modify train_vsl_agent function
def train_vsl_agent(data, episodes=100, batch_size=32):
    env = VSLEnvironment(data)
    state_size = len(env.state_features)
    action_size = len(env.speed_limits)
    agent = PPOAgent(state_size, action_size)
    
    # Dictionary to store results
    results = {
        'episode': [],
        'average_reward': [],
        'speed_limits': []
    }
    
    for episode in range(episodes):
        total_reward = 0
        state = env.reset()
        speed_limits_applied = []
        done = False
        
        while not done:
            # Act with action and log probability
            action, log_prob = agent.act(state), 0.0  # Default log_prob if not used
            next_state, reward, done, info = env.step(action)
            
            if next_state is None:  # End of data
                break
                
            speed_limits_applied.append(env.speed_limits[action])
            total_reward += reward
            
            # Store experience with log probability
            agent.remember(state, action, reward, next_state, done, log_prob)
            state = next_state
        
        # Train the agent on collected experiences
        if len(agent.memory) >= batch_size:
            agent.train(batch_size)
        
        # Store results
        results['episode'].append(episode)
        results['average_reward'].append(total_reward / len(data))
        results['speed_limits'].append(speed_limits_applied)
        
        print(f"Episode: {episode+1}/{episodes}, Avg Reward: {total_reward/len(data):.4f}")
    
    # Save the trained model
    agent.save("models/ppo_actor_model.weights.h5", "models/ppo_critic_model.weights.h5")
    return results, agent
# The rest of the code remains the same as in the original implementation
# (load_traffic_data, evaluate_vsl_agent, visualize_results, run_vsl_pipeline)
# Main training function

# Evaluate the trained agent
def evaluate_vsl_agent(data, agent):
    env = VSLEnvironment(data)
    state = env.reset()
    recommended_speeds = []
    actual_speeds = []
    flow_rates = []
    timestamps = []
    done = False
    
    while not done:
        action = agent.act(state)
        speed_limit = env.speed_limits[action]
        recommended_speeds.append(speed_limit)
        
        # Store actual values for comparison
        current_data = data.iloc[env.current_step]
        actual_speeds.append(current_data['average_speed_kmh'])
        flow_rates.append(current_data['flow_rate_vph'])
        timestamps.append(current_data['timestamp'])
        
        next_state, reward, done, _ = env.step(action)
        if next_state is None:
            break
        state = next_state
    
    return timestamps, recommended_speeds, actual_speeds, flow_rates

# Modified visualization function with animation
def visualize_results_with_animation(timestamps, recommended_speeds, actual_speeds, flow_rates):
    # Static visualization (same as before)
    plt.figure(figsize=(14, 8))
    
    # Plot recommended speed limits and actual speeds
    plt.subplot(2, 1, 1)
    plt.plot(timestamps, recommended_speeds, 'b-', label='Recommended Speed Limit')
    plt.plot(timestamps, actual_speeds, 'r-', label='Actual Average Speed')
    plt.ylabel('Speed (km/h)')
    plt.title('Continuous VSL Recommendations vs. Actual Speeds')
    plt.legend()
    plt.grid(True)
    
    # Plot flow rates
    plt.subplot(2, 1, 2)
    plt.plot(timestamps, flow_rates, 'g-', label='Flow Rate')
    plt.ylabel('Flow Rate (vph)')
    plt.xlabel('Time')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('output/vsl_results2.png')
    plt.close()
    
    # Create animated visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
    fig.suptitle('Variable Speed Limit System Animation', fontsize=16)
    
    # Format dates on x-axis
    date_format = DateFormatter('%H:%M')
    
    # Empty lines for initialization
    speed_limit_line, = ax1.plot([], [], 'b-', linewidth=2, label='Recommended Speed Limit')
    actual_speed_line, = ax1.plot([], [], 'r-', linewidth=2, label='Actual Average Speed')
    flow_rate_line, = ax2.plot([], [], 'g-', linewidth=2, label='Flow Rate')
    
    # Adding a vertical line to track current time
    current_time_line1 = ax1.axvline(x=timestamps[0], color='k', linestyle='--', alpha=0.5)
    current_time_line2 = ax2.axvline(x=timestamps[0], color='k', linestyle='--', alpha=0.5)
    
    # Adding a text annotation for the current speed limit
    speed_text = ax1.text(0.02, 0.92, '', transform=ax1.transAxes, fontsize=12, 
                         bbox=dict(facecolor='white', alpha=0.7))
    
    # Set axis labels and grid
    ax1.set_ylabel('Speed (km/h)')
    ax1.grid(True)
    ax1.legend(loc='upper right')
    
    ax2.set_ylabel('Flow Rate (vph)')
    ax2.set_xlabel('Time')
    ax2.grid(True)
    ax2.legend(loc='upper right')
    
    # Format the x-axis to display time correctly
    ax1.xaxis.set_major_formatter(date_format)
    ax2.xaxis.set_major_formatter(date_format)
    
    # Set fixed y-axis limits for better visualization
    ax1.set_ylim(0, max(max(recommended_speeds), max(actual_speeds)) * 1.1)
    ax2.set_ylim(0, max(flow_rates) * 1.1)
    
    # Set fixed x-axis limits
    ax1.set_xlim(timestamps[0], timestamps[-1])
    ax2.set_xlim(timestamps[0], timestamps[-1])
    
    # Create a counter for animation frames
    frame_count = len(timestamps)
    
    # Initialize the animation
    def init():
        speed_limit_line.set_data([], [])
        actual_speed_line.set_data([], [])
        flow_rate_line.set_data([], [])
        speed_text.set_text('')
        return speed_limit_line, actual_speed_line, flow_rate_line, current_time_line1, current_time_line2, speed_text
    
    # Animation function
    def animate(i):
        # Display progress for longer animations
        if i % 10 == 0:
            print(f"Animating frame {i+1}/{frame_count}")
        
        # Number of points to show (growing window)
        window_size = min(i + 1, len(timestamps))
        end_idx = i + 1
        
        # Update data
        speed_limit_line.set_data(timestamps[:end_idx], recommended_speeds[:end_idx])
        actual_speed_line.set_data(timestamps[:end_idx], actual_speeds[:end_idx])
        flow_rate_line.set_data(timestamps[:end_idx], flow_rates[:end_idx])
        
        # Update current time marker
        current_time = timestamps[i]
        current_time_line1.set_xdata([current_time, current_time])
        current_time_line2.set_xdata([current_time, current_time])
        
        # Update speed limit text
        speed_text.set_text(f'Current Speed Limit: {recommended_speeds[i]:.1f} km/h')
        
        return speed_limit_line, actual_speed_line, flow_rate_line, current_time_line1, current_time_line2, speed_text
    
    # Create animation
    print("Creating animation...")
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=frame_count, 
                                  interval=200, blit=True)
    
    # Save animation
    print("Saving animation (this may take a while)...")
    # Use a lower dpi for faster rendering during development, increase for final version
    anim.save('output/vsl_animation.mp4', writer='ffmpeg', fps=10, dpi=100)
    print("Animation saved to 'vsl_animation.mp4'")
    
    plt.close()
    
    # Additionally create a simplified GIF version (easier to view)
    # Use a subset of frames for a smaller file size
    frame_subset = max(1, frame_count // 100)  # Skip frames for efficiency
    
    fig, ax = plt.figure(figsize=(10, 6)), plt.axes()
    ax.set_title('VSL Speed Limit Animation')
    ax.set_ylabel('Speed (km/h)')
    ax.set_xlabel('Time')
    ax.grid(True)
    
    line, = ax.plot([], [], 'b-', linewidth=3, label='Speed Limit')
    time_text = ax.text(0.02, 0.92, '', transform=ax.transAxes, fontsize=12,
                       bbox=dict(facecolor='white', alpha=0.7))
    
    ax.set_xlim(timestamps[0], timestamps[-1])
    ax.set_ylim(min(recommended_speeds) * 0.9, max(recommended_speeds) * 1.1)
    ax.xaxis.set_major_formatter(date_format)
    ax.legend()
    
    def init_gif():
        line.set_data([], [])
        time_text.set_text('')
        return line, time_text
    
    def animate_gif(i):
        i = i * frame_subset  # Skip frames
        idx = min(i, len(timestamps)-1)
        line.set_data(timestamps[:idx+1], recommended_speeds[:idx+1])
        time_text.set_text(f'Time: {timestamps[idx].strftime("%H:%M")}\nSpeed: {recommended_speeds[idx]:.1f} km/h')
        return line, time_text
    
    gif_frames = len(timestamps) // frame_subset
    anim_gif = animation.FuncAnimation(fig, animate_gif, init_func=init_gif, 
                                     frames=gif_frames, interval=100, blit=True)
    
    print("Saving GIF animation...")
    anim_gif.save('output/vsl_animation.gif', writer='pillow', fps=5, dpi=80)
    print("GIF animation saved to 'vsl_animation.gif'")
    
    plt.close()



# Run the complete pipeline
def run_continuous_vsl_pipeline():
    # Load data
    data = load_traffic_data("traffic_data_2.csv")
    
    # Train the agent
    results, agent = train_vsl_agent(data, episodes=50)
    
    # Evaluate the agent
    timestamps, recommended_speeds, actual_speeds, flow_rates = evaluate_vsl_agent(data, agent)
    
    # Visualize results
    visualize_results_with_animation(timestamps, recommended_speeds, actual_speeds, flow_rates)
    
    return results

if __name__ == "__main__":
    results = run_continuous_vsl_pipeline()
    print("Continuous VSL implementation completed. Results and animations saved.")

Episode: 1/50, Avg Reward: 0.3884
Episode: 2/50, Avg Reward: 0.3605
Episode: 3/50, Avg Reward: 0.3848
Episode: 4/50, Avg Reward: 0.4060
Episode: 5/50, Avg Reward: 0.4087
Episode: 6/50, Avg Reward: 0.4069
Episode: 7/50, Avg Reward: 0.4085
Episode: 8/50, Avg Reward: 0.3645
Episode: 9/50, Avg Reward: 0.3369
Episode: 10/50, Avg Reward: 0.3714
Episode: 11/50, Avg Reward: 0.3289
Episode: 12/50, Avg Reward: 0.3766
Episode: 13/50, Avg Reward: 0.4130
Episode: 14/50, Avg Reward: 0.3757
Episode: 15/50, Avg Reward: 0.3538
Episode: 16/50, Avg Reward: 0.4013
Episode: 17/50, Avg Reward: 0.3347
Episode: 18/50, Avg Reward: 0.3657
Episode: 19/50, Avg Reward: 0.3566
Episode: 20/50, Avg Reward: 0.3759
Episode: 21/50, Avg Reward: 0.3339
Episode: 22/50, Avg Reward: 0.4141
Episode: 23/50, Avg Reward: 0.3572
Episode: 24/50, Avg Reward: 0.4076
Episode: 25/50, Avg Reward: 0.3556
Episode: 26/50, Avg Reward: 0.3544
Episode: 27/50, Avg Reward: 0.4473
Episode: 28/50, Avg Reward: 0.4230
Episode: 29/50, Avg Reward: 0

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Circle, Rectangle
from matplotlib.dates import DateFormatter
import matplotlib.font_manager as fm
from datetime import datetime
import time

# Continuous Variable Speed Limit Environment
class ContinuousVSLEnvironment:
    def __init__(self, data):
        self.data = data
        self.current_step = 0
        
        # Define continuous action space limits
        self.min_speed_limit = 50  # Minimum speed limit
        self.max_speed_limit = 140  # Maximum speed limit
        
        # Define state space features
        self.state_features = ['vehicle_count', 'average_speed_kmh', 'vehicle_density_vpkm']
        
        # Normalize the features for better learning
        self.feature_means = data[self.state_features].mean()
        self.feature_stds = data[self.state_features].std()
        
    def reset(self):
        self.current_step = 0
        return self._get_state()
    
    def _get_state(self):
        if self.current_step >= len(self.data):
            return None
        
        current_data = self.data.iloc[self.current_step][self.state_features]
        # Normalize the state
        normalized_state = (current_data - self.feature_means) / self.feature_stds
        return normalized_state.values.astype(np.float32)
    
    def step(self, action):
        if self.current_step >= len(self.data) - 1:
            return None, 0, True, {}
        
        # Scale continuous action to speed limit range
        scaled_speed_limit = self.min_speed_limit + (self.max_speed_limit - self.min_speed_limit) * action
        
        # Get current traffic conditions
        current_data = self.data.iloc[self.current_step]
        
        # Advance to the next step
        self.current_step += 1
        next_data = self.data.iloc[self.current_step]
        
        # Calculate reward based on traffic efficiency and safety
        reward = self._calculate_reward(current_data, next_data, scaled_speed_limit)
        
        # Get new state
        new_state = self._get_state()
        
        # Check if episode is done
        done = self.current_step >= len(self.data) - 1
        
        return new_state, reward, done, {'speed_limit': scaled_speed_limit}
    
    def _calculate_reward(self, current_data, next_data, speed_limit):
        # Traffic efficiency component: Reward higher flow rates
        flow_reward = next_data['flow_rate_vph'] / 1000  # Normalize
        
        # Safety component: Penalize if speed is much higher than the limit
        speed_compliance = max(0, 1 - max(0, next_data['average_speed_kmh'] - speed_limit) / 50) # 50 is min spped limit
        
        # Stability component: Reward lower variations in speed
        speed_stability = 1 / (1 + abs(next_data['average_speed_kmh'] - current_data['average_speed_kmh']))
        
        # Occupancy component: Reward balanced lane utilization
        occupancy_vars = np.var([
            next_data['occupancy_lane_1'], 
            next_data['occupancy_lane_2'], 
            next_data['occupancy_lane_3']
        ])
        occupancy_balance = 1 / (1 + occupancy_vars/1000)
        
        # Combined reward
        reward = (0.4 * flow_reward + 
                  0.3 * speed_compliance + 
                  0.2 * speed_stability + 
                  0.1 * occupancy_balance)
        
        return float(reward)  # Ensure float type

# Continuous PPO Agent
class ContinuousPPOAgent:
    def __init__(self, state_size):
        self.state_size = state_size
        
       # Improved hyperparameters
        self.learning_rate = 0.0001  # Lower learning rate for more stable learning
        self.gamma = 0.99  # Higher discount factor for better long-term planning
        self.epsilon_clip = 0.15  # Adjusted clipping parameter
        self.c1 = 0.8  # Value loss coefficient
        self.c2 = 0.02  # Entropy coefficient (slightly higher for better exploration)
        self.epochs = 5  # More epochs per update
        
        # Memory with larger capacity
        self.memory = []
        self.memory_capacity = 10000
        
        # Actor-Critic Network with improved architecture
        self.actor = self._build_actor_network()
        self.critic = self._build_critic_network()
        
        # Optimizers with learning rate decay
        self.actor_optimizer = keras.optimizers.Adam(
            learning_rate=keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=self.learning_rate,
                decay_steps=1000,
                decay_rate=0.95
            )
        )
        self.critic_optimizer = keras.optimizers.Adam(
            learning_rate=keras.optimizers.schedules.ExponentialDecay(
                initial_learning_rate=self.learning_rate,
                decay_steps=1000,
                decay_rate=0.95
            )
        )
    
    def _build_actor_network(self):
        # Actor network for policy (continuous action distribution)
        inputs = layers.Input(shape=(self.state_size,))
        
        # Add batch normalization for better training stability
        x = layers.BatchNormalization()(inputs)
        
        # Wider and deeper network
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(64, activation='relu')(x)


        # Output mean and log std for a Gaussian distribution
        mean = layers.Dense(1, activation='tanh')(x)  # Using tanh to bound the output
        mean = layers.Lambda(lambda x: x * 0.5 + 0.5)(mean)  # Rescale to [0,1]
        
        log_std = layers.Dense(1, activation='linear')(x)
        # Bound log_std for stability
        log_std = layers.Lambda(lambda x: tf.clip_by_value(x, -20, 0))(log_std)
        
        model = keras.Model(inputs=inputs, outputs=[mean, log_std])
        return model
    
    def _build_critic_network(self):
        # Critic network for value function estimation
        inputs = layers.Input(shape=(self.state_size,))
        
        # Add batch normalization
        x = layers.BatchNormalization()(inputs)
        
        # Wider and deeper network
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(64, activation='relu')(x)
        
        # Output a single value
        outputs = layers.Dense(1, activation='linear')(x)
        
        model = keras.Model(inputs=inputs, outputs=outputs)
        return model
    
    def remember(self, state, action, reward, next_state, done, old_mean, old_std):
        # Store experiences with additional information for PPO
        self.memory.append((state, action, reward, next_state, done, old_mean, old_std))
    
    def act(self, state):
        # Predict action distribution parameters
        state = np.array([state], dtype=np.float32)
        mean, log_std = self.actor.predict(state, verbose=0)
        mean, log_std = mean[0][0], log_std[0][0]
        
        # Sample action from Gaussian distribution
        std = np.exp(log_std)
        action = np.random.normal(mean, std)
        
        # Clip action to [0, 1] range for environment
        action = np.clip(action, 0, 1)
        
        return action, mean, std
    
    def train(self, batch_size):
        if len(self.memory) < batch_size:
            return
        
        # Prepare batches
        states = np.array([mem[0] for mem in self.memory], dtype=np.float32)
        actions = np.array([mem[1] for mem in self.memory], dtype=np.float32)
        rewards = np.array([mem[2] for mem in self.memory], dtype=np.float32)
        next_states = np.array([mem[3] for mem in self.memory], dtype=np.float32)
        dones = np.array([mem[4] for mem in self.memory])
        old_means = np.array([mem[5] for mem in self.memory], dtype=np.float32)
        old_stds = np.array([mem[6] for mem in self.memory], dtype=np.float32)
        
        # Compute returns and advantages
        returns = np.zeros_like(rewards)
        advantages = np.zeros_like(rewards)
        
        # Compute values
        values = self.critic.predict(states, verbose=0).flatten()
        next_values = self.critic.predict(next_states, verbose=0).flatten()
        
        # Compute returns and advantages
        for t in range(len(rewards)):
            # Compute TD error
            # Measures how much better an action was compared to the expected return.
            delta = rewards[t] + self.gamma * next_values[t] * (1 - dones[t]) - values[t]
            
            # Compute advantage
            advantages[t] = delta
            
            # Compute returns (GAE)
            # Generalized Advantage Estimation
            returns[t] = rewards[t] + self.gamma * next_values[t] * (1 - dones[t])
        
        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-10)
        
        # Perform PPO update
        self._ppo_update(states, actions, advantages, returns, old_means, old_stds)
        
        # Clear memory after training
        self.memory.clear()
    
    def _ppo_update(self, states, actions, advantages, returns, old_means, old_stds):
        # Use GradientTape for explicit gradient computation record for coompute the gradients
        # This function updates both the actor (policy) and critic (value function) networks.
        with tf.GradientTape() as actor_tape, tf.GradientTape() as critic_tape:
            # Actor network computations
            current_means, current_log_stds = self.actor(states)
            current_means = tf.squeeze(current_means)# remove unccessary dimensions
            current_log_stds = tf.squeeze(current_log_stds)
            current_stds = tf.exp(current_log_stds)
            
            # Compute log probabilities for continuous actions
            #log probability density function of a multivariate Gaussian distribution.
            log_prob = -0.5 * tf.math.log(2 * np.pi * current_stds**2) - \
                           0.5 * ((actions - current_means) / current_stds)**2
            log_prob = tf.reduce_sum(log_prob, axis=-1)
            
            # Compute old log probabilities
            old_log_prob = -0.5 * tf.math.log(2 * np.pi * old_stds**2) - \
                               0.5 * ((actions - old_means) / old_stds)**2
            old_log_prob = tf.reduce_sum(old_log_prob, axis=-1)
            
            # Compute probability ratio
            ratio = tf.exp(log_prob - old_log_prob)
            
            # Clipped surrogate objective
    #        PPO Clipping Mechanism:
    #       Limits how much the new policy can deviate from the old one.
    #       Prevents overly aggressive updates.
            surr1 = ratio * advantages
            surr2 = tf.clip_by_value(ratio, 1 - self.epsilon_clip, 1 + self.epsilon_clip) * advantages
            policy_loss = -tf.reduce_mean(tf.minimum(surr1, surr2))
            
            # Critic (value) loss
            # MSE Loss for the Critic Network:
            critic_values = tf.squeeze(self.critic(states))
            value_loss = tf.reduce_mean(tf.square(returns - critic_values))
            
            # Entropy loss (encourage exploration)
            entropy_loss = tf.reduce_mean(current_log_stds + 0.5 * np.log(2.0 * np.pi * np.e))
            
            # Composite losses
            # Actor loss: Policy loss with entropy regularization.
            #Critic loss: Value function loss.
            total_actor_loss = policy_loss - self.c2 * entropy_loss
            total_critic_loss = self.c1 * value_loss
        
        # Compute gradients
        actor_grads = actor_tape.gradient(total_actor_loss, self.actor.trainable_variables)
        critic_grads = critic_tape.gradient(total_critic_loss, self.critic.trainable_variables)
        
        # Clip gradients to prevent exploding gradients
        max_grad_norm = 0.5
        actor_grads = [tf.clip_by_norm(grad, max_grad_norm) for grad in actor_grads]
        critic_grads = [tf.clip_by_norm(grad, max_grad_norm) for grad in critic_grads]
        
        # Apply gradients
        # Updates the actor and critic networks.
        self.actor_optimizer.apply_gradients(zip(actor_grads, self.actor.trainable_variables))
        self.critic_optimizer.apply_gradients(zip(critic_grads, self.critic.trainable_variables))
    
    def save(self, actor_path, critic_path):
        self.actor.save_weights(actor_path)
        self.critic.save_weights(critic_path)
    
    def load(self, actor_path, critic_path):
        self.actor.load_weights(actor_path)
        self.critic.load_weights(critic_path)

# Load and prepare the data
def load_traffic_data(data_path):
    # In a real implementation, you'd load from a file
    df = pd.read_csv(data_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

# Training function for continuous VSL agent
def train_continuous_vsl_agent(data, episodes=100, batch_size=32):
    env = ContinuousVSLEnvironment(data)
    state_size = len(env.state_features)
    agent = ContinuousPPOAgent(state_size)
    
    # Dictionary to store results
    results = {
        'episode': [],
        'average_reward': [],
        'speed_limits': []
    }

     # Store best model
    best_reward = 0
    
    for episode in range(episodes):
        total_reward = 0
        state = env.reset()
        speed_limits_applied = []
        done = False
        
        while not done:
            # Act with action and distribution parameters
            action, mean, std = agent.act(state)
            next_state, reward, done, info = env.step(action)
            
            if next_state is None:  # End of data
                break
                
            # Scale action back to speed limit for recording
            scaled_speed_limit = env.min_speed_limit + (env.max_speed_limit - env.min_speed_limit) * action
            speed_limits_applied.append(scaled_speed_limit)
            
            total_reward += reward
            
            # Store experience with distribution parameters
            agent.remember(state, action, reward, next_state, done, mean, std)
            state = next_state
        
        # Train the agent on collected experiences
        if len(agent.memory) >= batch_size:
            agent.train(batch_size)
        
        # Store results
        avg_reward = total_reward / len(data)
        results['episode'].append(episode)
        results['average_reward'].append(avg_reward)
        results['speed_limits'].append(speed_limits_applied)

        # Save best model
        if avg_reward > best_reward:
            best_reward = avg_reward
            agent.save("models/best_ppo_actor_model.weights.h5", "models/best_ppo_critic_model.weights.h5")
            print(f"New best model saved with avg reward: {avg_reward:.4f}")
        
        print(f"Episode: {episode+1}/{episodes}, Avg Reward: {avg_reward:.4f}")
    
    # Save the trained model
    agent.save("models/continuous_ppo_actor_model.weights.h5", "models/continuous_ppo_critic_model.weights.h5")
    return results, agent

def create_speed_limit_panel_animation(timestamps, recommended_speeds, output_file='output/vsl_speed_panel.mp4'):
    """
    Create an animation of a digital speed limit panel that changes over time.
    
    Parameters:
    timestamps - List of datetime objects
    recommended_speeds - List of speed limit values
    output_file - Filename for the output animation
    """
    # Round speed limits to whole numbers for display
    speeds_rounded = [round(speed) for speed in recommended_speeds]
    
    # Set up the figure
    fig, ax = plt.subplots(figsize=(8, 10))
    plt.subplots_adjust(left=0, bottom=0, right=1, top=0.9)
    
    # Turn off the axis
    ax.axis('off')
    
    # Create a speed limit sign background
    def draw_speed_sign(speed):
        # Clear the axis for the new frame
        ax.clear()
        ax.axis('off')
        
        # Create the red circle border
        circle = Circle((0.5, 0.5), 0.4, facecolor='white', edgecolor='red', linewidth=10, transform=ax.transAxes)
        ax.add_patch(circle)
        
        # Add the speed text
        ax.text(0.5, 0.5, f"{speed}", fontsize=150, ha='center', va='center', 
                transform=ax.transAxes, fontweight='bold')
        
        # Add "km/h" text below
        ax.text(0.5, 0.25, "km/h", fontsize=40, ha='center', va='center', 
                transform=ax.transAxes)
        
        # Add timestamp
        time_idx = min(frame_num, len(timestamps)-1)
        current_time = timestamps[time_idx].strftime('%Y-%m-%d %H:%M:%S')
        ax.text(0.5, 0.9, f"Time: {current_time}", fontsize=20, ha='center', va='center',
                transform=ax.transAxes)
        
        # Add variable speed limit label
        ax.text(0.5, 0.85, "VARIABLE SPEED LIMIT", fontsize=20, ha='center', va='center',
                transform=ax.transAxes, fontweight='bold')
        
    # Initialize frame counter (used in the animation function)
    frame_num = 0
    
    # Animation function
    def update_sign(frame):
        nonlocal frame_num
        frame_num = frame
        
        # Get current speed (with protection against index errors)
        if frame < len(speeds_rounded):
            current_speed = speeds_rounded[frame]
        else:
            current_speed = speeds_rounded[-1]
            
        # Draw the updated sign
        draw_speed_sign(current_speed)
        
        # Print progress
        if frame % 10 == 0:
            print(f"Rendering frame {frame+1}/{len(speeds_rounded)}")
            
        return [ax]
    
    # Create animation
    print("Creating speed limit panel animation...")
    ani = animation.FuncAnimation(fig, update_sign, frames=len(speeds_rounded),
                                  interval=200, blit=False)
    
    # Save animation
    print(f"Saving animation to {output_file}...")
    ani.save(output_file, writer='ffmpeg', fps=5, dpi=100)
    plt.close(fig)
    print(f"Animation saved to {output_file}")
    
    # Create a GIF version as well (typically smaller file size)
    gif_output = output_file.replace('.mp4', '.gif')
    
    # Use fewer frames for the GIF to keep file size reasonable
    # We'll sample every nth frame
    frame_skip = max(1, len(speeds_rounded) // 50)
    frames_to_use = range(0, len(speeds_rounded), frame_skip)
    
    fig, ax = plt.subplots(figsize=(8, 10))
    plt.subplots_adjust(left=0, bottom=0, right=1, top=0.9)
    ax.axis('off')
    
    frame_num = 0
    
    # Create animation with reduced frames
    ani_gif = animation.FuncAnimation(fig, update_sign, frames=frames_to_use,
                                     interval=200, blit=False)
    
    print(f"Saving GIF animation to {gif_output}...")
    ani_gif.save(gif_output, writer='pillow', fps=2, dpi=80)
    plt.close(fig)
    print(f"GIF animation saved to {gif_output}")

# Evaluate the trained agent
def evaluate_continuous_vsl_agent(data, agent):
    env = ContinuousVSLEnvironment(data)
    state = env.reset()
    recommended_speeds = []
    actual_speeds = []
    flow_rates = []
    timestamps = []
    done = False
    
    while not done:
        action, _, _ = agent.act(state)
        # Scale action to speed limit
        speed_limit = env.min_speed_limit + (env.max_speed_limit - env.min_speed_limit) * action
        recommended_speeds.append(speed_limit)
        
        # Store actual values for comparison
        current_data = data.iloc[env.current_step]
        actual_speeds.append(current_data['average_speed_kmh'])
        flow_rates.append(current_data['flow_rate_vph'])
        timestamps.append(current_data['timestamp'])
        
        next_state, reward, done, _ = env.step(action)
        if next_state is None:
            break
        state = next_state

        
      # Create the speed panel animation
    create_speed_limit_panel_animation(timestamps, recommended_speeds)
    
    return timestamps, recommended_speeds, actual_speeds, flow_rates

# Modified visualization function with animation
def visualize_results_with_animation(timestamps, recommended_speeds, actual_speeds, flow_rates):
    # Static visualization (same as before)
    plt.figure(figsize=(14, 8))
    
    # Plot recommended speed limits and actual speeds
    plt.subplot(2, 1, 1)
    plt.plot(timestamps, recommended_speeds, 'b-', label='Recommended Speed Limit')
    plt.plot(timestamps, actual_speeds, 'r-', label='Actual Average Speed')
    plt.ylabel('Speed (km/h)')
    plt.title('Continuous VSL Recommendations vs. Actual Speeds')
    plt.legend()
    plt.grid(True)
    
    # Plot flow rates
    plt.subplot(2, 1, 2)
    plt.plot(timestamps, flow_rates, 'g-', label='Flow Rate')
    plt.ylabel('Flow Rate (vph)')
    plt.xlabel('Time')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('output/continuous_vsl_results.png')
    plt.close()
    

    
    plt.close()



# Run the complete pipeline
def run_continuous_vsl_pipeline():
    # Load data
    data = load_traffic_data("traffic_data_2.csv")
    
    # Train the agent
    results, agent = train_continuous_vsl_agent(data, episodes=50)
    
    # Evaluate the agent
    timestamps, recommended_speeds, actual_speeds, flow_rates = evaluate_continuous_vsl_agent(data, agent)
    
    # Visualize results
    visualize_results_with_animation(timestamps, recommended_speeds, actual_speeds, flow_rates)
    
    return results

if __name__ == "__main__":
    results = run_continuous_vsl_pipeline()
    print("Continuous VSL implementation completed. Results and animations saved.")

New best model saved with avg reward: 0.4061
Episode: 1/50, Avg Reward: 0.4061
New best model saved with avg reward: 0.4165
Episode: 2/50, Avg Reward: 0.4165
Episode: 3/50, Avg Reward: 0.3815
Episode: 4/50, Avg Reward: 0.2999
New best model saved with avg reward: 0.4423
Episode: 5/50, Avg Reward: 0.4423
Episode: 6/50, Avg Reward: 0.3358
Episode: 7/50, Avg Reward: 0.3376
Episode: 8/50, Avg Reward: 0.4336
Episode: 9/50, Avg Reward: 0.3625
Episode: 10/50, Avg Reward: 0.3842
Episode: 11/50, Avg Reward: 0.2924
Episode: 12/50, Avg Reward: 0.3376
Episode: 13/50, Avg Reward: 0.3223
Episode: 14/50, Avg Reward: 0.3978
Episode: 15/50, Avg Reward: 0.3456
Episode: 16/50, Avg Reward: 0.4023
Episode: 17/50, Avg Reward: 0.3699
Episode: 18/50, Avg Reward: 0.3585
Episode: 19/50, Avg Reward: 0.3785
Episode: 20/50, Avg Reward: 0.3606
Episode: 21/50, Avg Reward: 0.3933
Episode: 22/50, Avg Reward: 0.3394
Episode: 23/50, Avg Reward: 0.3776
New best model saved with avg reward: 0.4566
Episode: 24/50, Avg Rewa